In [1]:
import syft as sy

In [2]:
duet =  sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [4]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 9086c8694d384ecf837931ae00d56bf5>,[],,<class 'torch.Tensor'>
1,<UID: 53661b26c6c340a186de8fe76ca67ef9>,[],,<class 'torch.Tensor'>


In [5]:
data_pointer = duet.store[0]
target_pointer = duet.store[1]

In [11]:
in_dim = 64 
out_dim = 10
N_exemple = 1797

In [12]:
import torch 

In [15]:
class SyNet(sy.Module): 
    def __init__(self,torch_ref): 
        super(SyNet,self).__init__(torch_ref=torch_ref)
        self.layer1 = self.torch_ref.nn.Linear(in_dim,30)
        self.layer2 = self.torch_ref.nn.Linear(30 , 60)
        self.layer3 = self.torch_ref.nn.Linear(60,90)
        self.out = self.torch_ref.nn.Linear(90,out_dim)
    def forward(self,x):
        x = self.torch_ref.nn.functional.relu(self.layer1(x))
        x = self.torch_ref.nn.functional.relu(self.layer2(x))
        x = self.torch_ref.nn.functional.relu(self.layer3(x))
        output = self.torch_ref.nn.functional.log_softmax(self.out(x),dim=1)
        return output

In [16]:
model = SyNet(torch)

In [17]:
remote_model = model.send(duet)

In [18]:
remote_model

In [19]:
remote_torch = duet.torch

In [20]:
params = remote_model.parameters()
optim = remote_torch.optim.Adam(params=params,lr=0.1)

In [38]:
def train(iterations,model,torch_ref,optim,data_pointer,target_pointer): 
    losses=[]
    for i in range(iterations): 
        optim.zero_grad()
        output = model(data_pointer)
        loss = torch_ref.nn.functional.mse_loss(output,target_pointer)
        loss_item = loss.item()
        loss_value = loss_item.get(
            reason ="To evalute training process ",
            request_block = True,
            timeout_secs = 5
        )
        if i % 10 == 0: 
            print("Epoch ",i," loss ",loss_value)
        losses.append(loss_value)
        loss.backward()
        optim.step()
    return losses

In [39]:
iterations = 1
losses = train(iterations,remote_model,remote_torch,optim,data_pointer,target_pointer)

Epoch  0  loss  None


In [40]:
type(data_pointer)

syft.proxy.torch.TensorPointer